In [1]:
#Imports


import pandas as pd
from pandas import json_normalize
import requests 
import json
import os
from dotenv import load_dotenv #conda install -c conda-forge python-dotenv
import time
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from geopy.distance import geodesic as GD

import src.functions as functions

In [2]:
import os
load_dotenv()
key = os.getenv("4squarekey")

In [3]:
# I want the companies from City 1: Barcelona
# I want the companies from City 2: Amsterdam
# I want the companies from City 3: San Francisco

# I will scrape them from https://gamedevmap.com/index.php?location={CITYNAME}

In [4]:
"""import requests

url = "https://api.foursquare.com/v3/places/nearby?ll=41.3926679%2C2.1401891&query=2%20Awesome%20Studio"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3+fZhukazIdsdQYC8kjNOyqoI3tfZlCBgktME1Z2NElo="
}

response = requests.get(url, headers=headers).json()

response"""

'import requests\n\nurl = "https://api.foursquare.com/v3/places/nearby?ll=41.3926679%2C2.1401891&query=2%20Awesome%20Studio"\n\nheaders = {\n    "accept": "application/json",\n    "Authorization": "fsq3+fZhukazIdsdQYC8kjNOyqoI3tfZlCBgktME1Z2NElo="\n}\n\nresponse = requests.get(url, headers=headers).json()\n\nresponse'

In [5]:
## REWORK, I WILL LOOK FOR THE COMPANIES USING MONGO DB

In [6]:
# Import the database from Mongo
from pymongo import MongoClient
import pandas as pd
import time
client = MongoClient("localhost:27017")
client.list_database_names()
db = client["ironhack"]
database = "ironhack"
collection = "companies"
c = db.get_collection("companies")

In [7]:
result = c.find_one()
result.keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [8]:
gaming = list(c.find({"category_code": {"$regex": "game"}}, {"_id":0, "name":1, "offices.latitude":1,"offices.longitude":1,"offices.city":1,"offices.country_code":1}))



In [9]:
gaming

[{'name': 'Flektor',
  'offices': [{'city': 'Culver City',
    'country_code': 'USA',
    'latitude': 34.025958,
    'longitude': -118.379768}]},
 {'name': 'Lala',
  'offices': [{'city': 'Palo Alto',
    'country_code': 'USA',
    'latitude': 37.451151,
    'longitude': -122.154369}]},
 {'name': 'Joost',
  'offices': [{'city': 'New York',
    'country_code': 'USA',
    'latitude': 40.7464969,
    'longitude': -74.0094471}]},
 {'name': 'Babelgum',
  'offices': [{'city': 'London',
    'country_code': 'GBR',
    'latitude': 53.344104,
    'longitude': -6.267494}]},
 {'name': 'Sparter',
  'offices': [{'city': None,
    'country_code': 'USA',
    'latitude': 37.09024,
    'longitude': -95.712891}]},
 {'name': 'Kyte',
  'offices': [{'city': 'San Francisco',
    'country_code': 'USA',
    'latitude': 37.788482,
    'longitude': -122.409173}]},
 {'name': 'Veoh',
  'offices': [{'city': 'San Diego',
    'country_code': 'USA',
    'latitude': 32.902266,
    'longitude': -117.20834}]},
 {'name': '

In [10]:

df = pd.DataFrame(gaming)

In [11]:
df["offices"].shape[0]



1083

In [12]:
df["offices"][2][0]

{'city': 'New York',
 'country_code': 'USA',
 'latitude': 40.7464969,
 'longitude': -74.0094471}

In [13]:
df

,name,offices
0,Flektor,"[{'city': 'Culver City', 'country_code': 'USA'..."
1,Lala,"[{'city': 'Palo Alto', 'country_code': 'USA', ..."
2,Joost,"[{'city': 'New York', 'country_code': 'USA', '..."
3,Babelgum,"[{'city': 'London', 'country_code': 'GBR', 'la..."
4,Sparter,"[{'city': None, 'country_code': 'USA', 'latitu..."
...,...,...
1078,Symposium Productions,[]
1079,Rev Media,"[{'city': 'Dallas', 'country_code': 'USA', 'la..."
1080,Fictionaut,"[{'city': 'Astoria', 'country_code': 'USA', 'l..."
1081,Tiny Speck,"[{'city': 'Vancouver', 'country_code': 'CAN', ..."


In [14]:
city = []
country = []
latitude = []
longitude = []
i=0
while i < (df["offices"].shape[0]):
    if df["offices"][i] == []:
        city.append(np.nan)
        country.append(np.nan)
        latitude.append(np.nan)
        longitude.append(np.nan)
    else:    
        city.append(df["offices"][i][0]["city"])
        country.append(df["offices"][i][0]["country_code"])
        latitude.append(df["offices"][i][0]["latitude"])
        longitude.append(df["offices"][i][0]["longitude"])
    i+=1
df["city"] = city
df["country"] = country
df["latitude"] = latitude
df["longitude"] = longitude
df



,name,offices,city,country,latitude,longitude
0,Flektor,"[{'city': 'Culver City', 'country_code': 'USA'...",Culver City,USA,34.025958,-118.379768
1,Lala,"[{'city': 'Palo Alto', 'country_code': 'USA', ...",Palo Alto,USA,37.451151,-122.154369
2,Joost,"[{'city': 'New York', 'country_code': 'USA', '...",New York,USA,40.746497,-74.009447
3,Babelgum,"[{'city': 'London', 'country_code': 'GBR', 'la...",London,GBR,53.344104,-6.267494
4,Sparter,"[{'city': None, 'country_code': 'USA', 'latitu...",None,USA,37.090240,-95.712891
...,...,...,...,...,...,...
1078,Symposium Productions,[],NaN,NaN,NaN,NaN
1079,Rev Media,"[{'city': 'Dallas', 'country_code': 'USA', 'la...",Dallas,USA,NaN,NaN
1080,Fictionaut,"[{'city': 'Astoria', 'country_code': 'USA', 'l...",Astoria,USA,40.759795,-73.916127
1081,Tiny Speck,"[{'city': 'Vancouver', 'country_code': 'CAN', ...",Vancouver,CAN,49.282455,-123.109217


In [15]:
df.drop(columns=["offices"],inplace=True)
df.dropna(inplace=True)
df

,name,city,country,latitude,longitude
0,Flektor,Culver City,USA,34.025958,-118.379768
1,Lala,Palo Alto,USA,37.451151,-122.154369
2,Joost,New York,USA,40.746497,-74.009447
3,Babelgum,London,GBR,53.344104,-6.267494
5,Kyte,San Francisco,USA,37.788482,-122.409173
...,...,...,...,...,...
1072,DJ Nitrogen,San Francisco,USA,37.784174,-122.449408
1073,Shape Games,Alexandria,USA,38.814208,-77.042423
1080,Fictionaut,Astoria,USA,40.759795,-73.916127
1081,Tiny Speck,Vancouver,CAN,49.282455,-123.109217


In [16]:
df = df[df.groupby("city").city.transform("count")>1]
df = df[df.groupby("country").country.transform("count")>1]


In [17]:
df

,name,city,country,latitude,longitude
0,Flektor,Culver City,USA,34.025958,-118.379768
1,Lala,Palo Alto,USA,37.451151,-122.154369
2,Joost,New York,USA,40.746497,-74.009447
3,Babelgum,London,GBR,53.344104,-6.267494
5,Kyte,San Francisco,USA,37.788482,-122.409173
...,...,...,...,...,...
1065,Film Annex,New York,USA,40.746497,-74.009447
1067,FremantleMedia Ventures,New York,USA,40.743945,-73.985542
1071,MechaWorks,Sunnyvale,USA,37.383444,-122.009405
1072,DJ Nitrogen,San Francisco,USA,37.784174,-122.449408


In [18]:
df.reset_index(inplace=True)
df

,index,name,city,country,latitude,longitude
0,0,Flektor,Culver City,USA,34.025958,-118.379768
1,1,Lala,Palo Alto,USA,37.451151,-122.154369
2,2,Joost,New York,USA,40.746497,-74.009447
3,3,Babelgum,London,GBR,53.344104,-6.267494
4,5,Kyte,San Francisco,USA,37.788482,-122.409173
...,...,...,...,...,...,...
409,1065,Film Annex,New York,USA,40.746497,-74.009447
410,1067,FremantleMedia Ventures,New York,USA,40.743945,-73.985542
411,1071,MechaWorks,Sunnyvale,USA,37.383444,-122.009405
412,1072,DJ Nitrogen,San Francisco,USA,37.784174,-122.449408


In [19]:
df.drop(columns=["index"],inplace=True)

In [20]:
df

,name,city,country,latitude,longitude
0,Flektor,Culver City,USA,34.025958,-118.379768
1,Lala,Palo Alto,USA,37.451151,-122.154369
2,Joost,New York,USA,40.746497,-74.009447
3,Babelgum,London,GBR,53.344104,-6.267494
4,Kyte,San Francisco,USA,37.788482,-122.409173
...,...,...,...,...,...
409,Film Annex,New York,USA,40.746497,-74.009447
410,FremantleMedia Ventures,New York,USA,40.743945,-73.985542
411,MechaWorks,Sunnyvale,USA,37.383444,-122.009405
412,DJ Nitrogen,San Francisco,USA,37.784174,-122.449408


In [21]:
design = list(c.find({"tag_list":{"$regex": "design"}}, {"_id":0, "name":1,"offices.city":1,"offices.country_code":1}))

designdf = pd.DataFrame(design)
designdf

,name,offices
0,Digg,"[{'city': 'San Francisco', 'country_code': 'US..."
1,Adobe Systems,"[{'city': 'San Jose', 'country_code': 'USA'}]"
2,Negonation,"[{'city': '', 'country_code': 'ESP'}]"
3,GENWI,"[{'city': 'Los Altos', 'country_code': 'USA'}]"
4,Aviary,"[{'city': 'NY', 'country_code': 'USA'}]"
...,...,...
650,Top Rated Website Designers,"[{'city': 'Carlsbad', 'country_code': 'USA'}, ..."
651,United On-Line,"[{'city': 'Galatsi, Attiki', 'country_code': '..."
652,ITERNOVA,"[{'city': 'Teruel', 'country_code': 'ESP'}, {'..."
653,Atlantic BT,"[{'city': 'Raleigh', 'country_code': 'USA'}]"


In [22]:
city = []
for i in range(designdf.shape[0]):
    if designdf["offices"][i] == []:
        city.append(np.nan)
    else:
        city.append(designdf["offices"][i][0]["city"])
designdf["city"] = city
designdf.drop(columns=["offices"],inplace=True)

In [23]:
design_by_city = designdf["city"].value_counts()
design_by_city = pd.DataFrame(design_by_city)
design_by_city

,city
London,16
New York,15
,15
San Francisco,12
Bangalore,9
...,...
Boiling Springs,1
Sidlamafa,1
Hopkinton,1
Melrose,1


In [24]:
design_by_city.reset_index(inplace=True)

In [25]:
design_by_city

,index,city
0,London,16
1,New York,15
2,,15
3,San Francisco,12
4,Bangalore,9
...,...,...
337,Boiling Springs,1
338,Sidlamafa,1
339,Hopkinton,1
340,Melrose,1


In [26]:

df=df.merge(design_by_city, left_on='city', right_on='index')
df.rename(columns={"city_x": "city", "city_y": "nº design comp"},inplace=True)
df.drop(columns="index", inplace=True)
df


,name,city,country,latitude,longitude,nº design comp
0,Lala,Palo Alto,USA,37.451151,-122.154369,6
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6
2,OnLive,Palo Alto,USA,37.090240,-95.712891,6
3,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6
4,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6
...,...,...,...,...,...,...
327,Panfu,Berlin,DEU,52.528311,13.412099,2
328,Jakun Media,Berlin,DEU,53.584935,10.125143,2
329,Sauspiel,Berlin,DEU,52.494307,13.434474,2
330,Preview Networks,Copenhagen,DNK,55.665981,12.580075,1


In [27]:
# tech startup with cap > $1M
cond1 = {"tag_list":{"$regex": "tech"}}
cond2= {"total_money_raised":{"$regex":"M|B"}}
condi = [cond1,cond2]
tech = list(c.find({"$and":condi}, {"_id":0, "name":1,"offices.city":1,"total_money_raised":1}))
techdf = pd.DataFrame(tech)

In [28]:
techdf.shape[0]

153

In [29]:
techdf["offices"][152][0]["city"].title()

'London'

In [30]:
o=[]
for i in range(techdf.shape[0]):
    o.append(techdf["offices"][i][0]["city"].title())

techdf["offices"] = o

In [31]:
techdf

,name,total_money_raised,offices
0,Digg,$45M,San Francisco
1,Powerset,$22.5M,San Francisco
2,CastTV,$3.1M,San Francisco
3,eBuddy,€11.5M,Amsterdam
4,Grockit,$44.7M,San Francisco
...,...,...,...
148,Inspire Medical Systems,$17M,Maple Grove
149,DeciZium,$1.5M,San Jose
150,StyleCaster,$5M,New York City
151,Clavis Technology,€3.66M,Dublin


In [32]:
techdf_city = techdf["offices"].value_counts()
techdf_city = pd.DataFrame(techdf_city)
techdf_city.reset_index(inplace=True)
techdf_city

,index,offices
0,San Francisco,24
1,New York,8
2,Austin,8
3,Palo Alto,4
4,Sunnyvale,4
...,...,...
79,Mclean,1
80,Columbus,1
81,Mechelen,1
82,Portland,1


In [33]:
df

,name,city,country,latitude,longitude,nº design comp
0,Lala,Palo Alto,USA,37.451151,-122.154369,6
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6
2,OnLive,Palo Alto,USA,37.090240,-95.712891,6
3,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6
4,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6
...,...,...,...,...,...,...
327,Panfu,Berlin,DEU,52.528311,13.412099,2
328,Jakun Media,Berlin,DEU,53.584935,10.125143,2
329,Sauspiel,Berlin,DEU,52.494307,13.434474,2
330,Preview Networks,Copenhagen,DNK,55.665981,12.580075,1


In [34]:
df=df.merge(techdf_city, left_on='city', right_on='index')
df.rename(columns={"city_x": "city", "offices": "nº tech (>$1M)"},inplace=True)
df.drop(columns="index", inplace=True)
df

,name,city,country,latitude,longitude,nº design comp,nº tech (>$1M)
0,Lala,Palo Alto,USA,37.451151,-122.154369,6,4
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6,4
2,OnLive,Palo Alto,USA,37.090240,-95.712891,6,4
3,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6,4
4,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6,4
...,...,...,...,...,...,...,...
227,Blizzard Entertainment,Irvine,USA,33.669500,-117.822200,1,1
228,DanceShowoff,Irvine,USA,33.671428,-117.791423,1,1
229,Panfu,Berlin,DEU,52.528311,13.412099,2,1
230,Jakun Media,Berlin,DEU,53.584935,10.125143,2,1


In [35]:
# I want to find the coord of the closest - #LIMIT 1!!!!!

query = "starbucks"
starb=[]
for a in zip(df["latitude"],df["longitude"]):
    starb.append(functions.get_query_dist(query,a[0],a[1]))


In [36]:
starbuck_dist = functions.clean_coord(starb)


In [37]:
df["dist to starbucks (m)"]= starbuck_dist
# I will drop NaN

In [38]:
df.dropna(inplace=True)
df

,name,city,country,latitude,longitude,nº design comp,nº tech (>$1M),dist to starbucks (m)
0,Lala,Palo Alto,USA,37.451151,-122.154369,6,4,729.0
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6,4,2312.0
3,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6,4,396.0
4,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6,4,1046.0
5,Mytopia,Palo Alto,USA,34.007967,-118.489681,6,4,357.0
...,...,...,...,...,...,...,...,...
227,Blizzard Entertainment,Irvine,USA,33.669500,-117.822200,1,1,743.0
228,DanceShowoff,Irvine,USA,33.671428,-117.791423,1,1,193.0
229,Panfu,Berlin,DEU,52.528311,13.412099,2,1,863.0
230,Jakun Media,Berlin,DEU,53.584935,10.125143,2,1,9445.0


In [39]:
# I will look now for the distance to the closest airport --> function get category distance!!
query = "airport"
dist_to_airport =[]
for index,row in df.iterrows():

        dist_to_airport.append(functions.get_query_dist(query,row["latitude"],row["longitude"]))
        

df["dist to airport (km)"] = functions.clean_coord(dist_to_airport)
df["dist to airport (km)"]=[x/1000 for x in df["dist to airport (km)"]]
df.dropna(inplace=True)
df


,name,city,country,latitude,longitude,nº design comp,nº tech (>$1M),dist to starbucks (m),dist to airport (km)
0,Lala,Palo Alto,USA,37.451151,-122.154369,6,4,729.0,27.550
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6,4,2312.0,30.953
3,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6,4,396.0,27.437
4,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6,4,1046.0,28.251
5,Mytopia,Palo Alto,USA,34.007967,-118.489681,6,4,357.0,10.506
...,...,...,...,...,...,...,...,...,...
224,BAM Studios,Chicago,USA,41.893622,-87.627423,8,2,231.0,24.437
225,Plugola,Chicago,USA,41.907542,-87.662990,8,2,540.0,21.131
226,Midway Games,Chicago,USA,41.879535,-87.624333,8,2,25.0,25.302
227,Blizzard Entertainment,Irvine,USA,33.669500,-117.822200,1,1,743.0,3.813


In [40]:
# I want to get the number of night clubs nearby my location --> nearby foursquare


In [41]:
## afegir a les funcions, que venen de functions: get_multiquery_dist i get_avg_distance_from_list
dist_to_clubs =[]
for index,row in df.iterrows():
    dist_to_clubs.append(functions.get_multiquery_dist("club",row["latitude"],row["longitude"]))
avg_dis = functions.get_avg_distance_from_list(dist_to_clubs)
df["average dist to clubs (km)"] = [x/1000 for x in avg_dis]

In [42]:
df

,name,city,country,latitude,longitude,nº design comp,nº tech (>$1M),dist to starbucks (m),dist to airport (km),average dist to clubs (km)
0,Lala,Palo Alto,USA,37.451151,-122.154369,6,4,729.0,27.550,1.678444
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6,4,2312.0,30.953,3.469333
3,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6,4,396.0,27.437,0.995125
4,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6,4,1046.0,28.251,1.400000
5,Mytopia,Palo Alto,USA,34.007967,-118.489681,6,4,357.0,10.506,1.598714
...,...,...,...,...,...,...,...,...,...,...
224,BAM Studios,Chicago,USA,41.893622,-87.627423,8,2,231.0,24.437,0.651333
225,Plugola,Chicago,USA,41.907542,-87.662990,8,2,540.0,21.131,1.614700
226,Midway Games,Chicago,USA,41.879535,-87.624333,8,2,25.0,25.302,0.777222
227,Blizzard Entertainment,Irvine,USA,33.669500,-117.822200,1,1,743.0,3.813,2.669778


In [43]:
dist_to_vegan =[]
for index,row in df.iterrows():
    dist_to_vegan.append(functions.get_multiquery_dist("vegan restaurant",row["latitude"],row["longitude"]))
avg_dis = functions.get_avg_distance_from_list(dist_to_vegan)
df["average dist to vegan rest. (km)"] = [x/1000 for x in avg_dis]


#= [x/1000 for x in dist_to_vegan]

In [44]:
df

,name,city,country,latitude,longitude,nº design comp,nº tech (>$1M),dist to starbucks (m),dist to airport (km),average dist to clubs (km),average dist to vegan rest. (km)
0,Lala,Palo Alto,USA,37.451151,-122.154369,6,4,729.0,27.550,1.678444,8.588200
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6,4,2312.0,30.953,3.469333,12.115556
3,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6,4,396.0,27.437,0.995125,9.098222
4,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6,4,1046.0,28.251,1.400000,9.678000
5,Mytopia,Palo Alto,USA,34.007967,-118.489681,6,4,357.0,10.506,1.598714,3.441222
...,...,...,...,...,...,...,...,...,...,...,...
224,BAM Studios,Chicago,USA,41.893622,-87.627423,8,2,231.0,24.437,0.651333,0.612875
225,Plugola,Chicago,USA,41.907542,-87.662990,8,2,540.0,21.131,1.614700,1.489750
226,Midway Games,Chicago,USA,41.879535,-87.624333,8,2,25.0,25.302,0.777222,0.391857
227,Blizzard Entertainment,Irvine,USA,33.669500,-117.822200,1,1,743.0,3.813,2.669778,5.932900


In [45]:
schools_nearby =[]
for index,row in df.iterrows():
    schools_nearby.append(functions.get_multiquery_dist("school",row["latitude"],row["longitude"]))
avg_dis = functions.get_avg_distance_from_list(schools_nearby)
schools_nearby_average= [x/1000 for x in avg_dis]

In [46]:
df["average dist to schools (km)"]=schools_nearby_average

In [47]:
df.dropna(inplace=True)
df

,name,city,country,latitude,longitude,nº design comp,nº tech (>$1M),dist to starbucks (m),dist to airport (km),average dist to clubs (km),average dist to vegan rest. (km),average dist to schools (km)
0,Lala,Palo Alto,USA,37.451151,-122.154369,6,4,729.0,27.550,1.678444,8.588200,2.012100
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6,4,2312.0,30.953,3.469333,12.115556,2.536900
3,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6,4,396.0,27.437,0.995125,9.098222,1.855889
4,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6,4,1046.0,28.251,1.400000,9.678000,1.610500
5,Mytopia,Palo Alto,USA,34.007967,-118.489681,6,4,357.0,10.506,1.598714,3.441222,1.091200
...,...,...,...,...,...,...,...,...,...,...,...,...
224,BAM Studios,Chicago,USA,41.893622,-87.627423,8,2,231.0,24.437,0.651333,0.612875,0.713300
225,Plugola,Chicago,USA,41.907542,-87.662990,8,2,540.0,21.131,1.614700,1.489750,1.250700
226,Midway Games,Chicago,USA,41.879535,-87.624333,8,2,25.0,25.302,0.777222,0.391857,0.488400
227,Blizzard Entertainment,Irvine,USA,33.669500,-117.822200,1,1,743.0,3.813,2.669778,5.932900,2.634100


In [48]:
dist_basket =[]
for index,row in df.iterrows():
    dist_basket.append(functions.get_multiquery_dist("basketball stadium",row["latitude"],row["longitude"]))

min_dist_basket=[min(x)/1000 for x in dist_basket]


In [49]:
df["dist. to closes basketball court (km)"] = min_dist_basket

In [50]:
df

,name,city,country,latitude,longitude,nº design comp,nº tech (>$1M),dist to starbucks (m),dist to airport (km),average dist to clubs (km),average dist to vegan rest. (km),average dist to schools (km),dist. to closes basketball court (km)
0,Lala,Palo Alto,USA,37.451151,-122.154369,6,4,729.0,27.550,1.678444,8.588200,2.012100,1.917
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6,4,2312.0,30.953,3.469333,12.115556,2.536900,2.523
3,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6,4,396.0,27.437,0.995125,9.098222,1.855889,1.115
4,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6,4,1046.0,28.251,1.400000,9.678000,1.610500,0.304
5,Mytopia,Palo Alto,USA,34.007967,-118.489681,6,4,357.0,10.506,1.598714,3.441222,1.091200,0.777
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,BAM Studios,Chicago,USA,41.893622,-87.627423,8,2,231.0,24.437,0.651333,0.612875,0.713300,0.891
225,Plugola,Chicago,USA,41.907542,-87.662990,8,2,540.0,21.131,1.614700,1.489750,1.250700,0.964
226,Midway Games,Chicago,USA,41.879535,-87.624333,8,2,25.0,25.302,0.777222,0.391857,0.488400,0.565
227,Blizzard Entertainment,Irvine,USA,33.669500,-117.822200,1,1,743.0,3.813,2.669778,5.932900,2.634100,0.587


In [51]:
# closest dog salon!
dist_dog =[]
for index,row in df.iterrows():
    dist_dog.append(functions.get_multiquery_dist("pet grooming",row["latitude"],row["longitude"]))

df["dist. to closest canine salon (km)"]=[min(x)/1000 for x in dist_dog]


In [53]:
df.dropna(inplace=True)
df

,name,city,country,latitude,longitude,nº design comp,nº tech (>$1M),dist to starbucks (m),dist to airport (km),average dist to clubs (km),average dist to vegan rest. (km),average dist to schools (km),dist. to closes basketball court (km),dist. to closest canine salon (km)
0,Lala,Palo Alto,USA,37.451151,-122.154369,6,4,729.0,27.550,1.678444,8.588200,2.012100,1.917,6.491
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6,4,2312.0,30.953,3.469333,12.115556,2.536900,2.523,6.842
3,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6,4,396.0,27.437,0.995125,9.098222,1.855889,1.115,5.767
4,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6,4,1046.0,28.251,1.400000,9.678000,1.610500,0.304,6.332
5,Mytopia,Palo Alto,USA,34.007967,-118.489681,6,4,357.0,10.506,1.598714,3.441222,1.091200,0.777,1.982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,BAM Studios,Chicago,USA,41.893622,-87.627423,8,2,231.0,24.437,0.651333,0.612875,0.713300,0.891,0.935
225,Plugola,Chicago,USA,41.907542,-87.662990,8,2,540.0,21.131,1.614700,1.489750,1.250700,0.964,5.169
226,Midway Games,Chicago,USA,41.879535,-87.624333,8,2,25.0,25.302,0.777222,0.391857,0.488400,0.565,1.560
227,Blizzard Entertainment,Irvine,USA,33.669500,-117.822200,1,1,743.0,3.813,2.669778,5.932900,2.634100,0.587,4.248


In [54]:
# Cost of living
#https://www.numbeo.com/cost-of-living/rankings.jsp
a = df["city"].unique()
a

array(['Palo Alto', 'New York', 'London', 'San Francisco', 'San Mateo',
       'Burlingame', 'Seattle', 'Mountain View', 'Boston', 'Santa Monica',
       'Toronto', 'San Jose', 'Santa Clara', 'Paris', 'Amsterdam',
       'Cambridge', 'Austin', 'Montreal', 'Carlsbad', 'Vancouver',
       'Munich', 'Chicago', 'Irvine'], dtype=object)

In [55]:
#I will scrape the website to get - City + Cost of living
url = "https://www.numbeo.com/cost-of-living/rankings.jsp"
class_= "stripe"
temp = functions.scrape_table(url, "table", class_)
temp2 = temp[[3,5]]
city = []
for i in temp2[3]:
    i=i.split(">")
    city.append(i[1][:-4])
city

cost=[]
for i in temp2[5]:
    i = i.split(">")
    cost.append(i[1][:-4])
cost_living_df = pd.DataFrame({"city":city, "cost living index":cost})
cost_living_df

,city,cost living index
0,"Hamilton, Bermuda",145.98
1,"Basel, Switzerland",119.25
2,"Zurich, Switzerland",118.67
3,"Zug, Switzerland",117.97
4,"Lausanne, Switzerland",117.11
...,...,...
505,"Islamabad, Pakistan",18.66
506,"Rawalpindi, Pakistan",18.00
507,"Multan, Pakistan",16.92
508,"Faisalabad, Pakistan",16.86


In [56]:
cost_living_df ["city"] = [x.split(",")[0] for x in cost_living_df ["city"]]
cost_living_df

,city,cost living index
0,Hamilton,145.98
1,Basel,119.25
2,Zurich,118.67
3,Zug,117.97
4,Lausanne,117.11
...,...,...
505,Islamabad,18.66
506,Rawalpindi,18.00
507,Multan,16.92
508,Faisalabad,16.86


In [57]:
a = df["city"].unique()
a  

array(['Palo Alto', 'New York', 'London', 'San Francisco', 'San Mateo',
       'Burlingame', 'Seattle', 'Mountain View', 'Boston', 'Santa Monica',
       'Toronto', 'San Jose', 'Santa Clara', 'Paris', 'Amsterdam',
       'Cambridge', 'Austin', 'Montreal', 'Carlsbad', 'Vancouver',
       'Munich', 'Chicago', 'Irvine'], dtype=object)

In [58]:
cost_living_df.loc[cost_living_df["city"] == "San Diego"]

,city,cost living index
38,San Diego,78.41


In [59]:
# The neighbourhoods will have same cost of living as the main city-->> i will add it to the df to join it!
new_row1 = {'city':'Palo Alto', 'cost living index':95.43}
new_row2 = {'city':'San Mateo', 'cost living index':95.43}
new_row3 = {'city':'Burlingame', 'cost living index':95.43}

new_row5 = {'city':'Mountain View', 'cost living index':95.43}
new_row6 = {'city':'Santa Monica', 'cost living index':77.65}
new_row7 = {'city':'Cambridge', 'cost living index':83.71}
new_row8 = {'city':'Carlsbad', 'cost living index':78.41}

#append row to the dataframe
cost_living_df = cost_living_df.append(new_row1, ignore_index=True)
cost_living_df = cost_living_df.append(new_row2, ignore_index=True)
cost_living_df = cost_living_df.append(new_row3, ignore_index=True)
cost_living_df = cost_living_df.append(new_row5, ignore_index=True)
cost_living_df = cost_living_df.append(new_row6, ignore_index=True)
cost_living_df = cost_living_df.append(new_row7, ignore_index=True)
cost_living_df = cost_living_df.append(new_row8, ignore_index=True)
cost_living_df 

/var/folders/7s/v9wty4qj3t1cw493y3p29dbc0000gn/T/ipykernel_55181/3866593058.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_living_df = cost_living_df.append(new_row1, ignore_index=True)
/var/folders/7s/v9wty4qj3t1cw493y3p29dbc0000gn/T/ipykernel_55181/3866593058.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_living_df = cost_living_df.append(new_row2, ignore_index=True)
/var/folders/7s/v9wty4qj3t1cw493y3p29dbc0000gn/T/ipykernel_55181/3866593058.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cost_living_df = cost_living_df.append(new_row3, ignore_index=True)
/var/folders/7s/v9wty4qj3t1cw493y3p29dbc0000gn/T/ipykernel_55181/3866593058.py:15: FutureWarning: The frame.append method is deprecated and wi

,city,cost living index
0,Hamilton,145.98
1,Basel,119.25
2,Zurich,118.67
3,Zug,117.97
4,Lausanne,117.11
...,...,...
512,Burlingame,95.43
513,Mountain View,95.43
514,Santa Monica,77.65
515,Cambridge,83.71


In [60]:
cost_living_df

,city,cost living index
0,Hamilton,145.98
1,Basel,119.25
2,Zurich,118.67
3,Zug,117.97
4,Lausanne,117.11
...,...,...
512,Burlingame,95.43
513,Mountain View,95.43
514,Santa Monica,77.65
515,Cambridge,83.71


In [61]:

df=df.merge(cost_living_df, left_on='city', right_on='city')
df

,name,city,country,latitude,longitude,nº design comp,nº tech (>$1M),dist to starbucks (m),dist to airport (km),average dist to clubs (km),average dist to vegan rest. (km),average dist to schools (km),dist. to closes basketball court (km),dist. to closest canine salon (km),cost living index
0,Lala,Palo Alto,USA,37.451151,-122.154369,6,4,729.0,27.550,1.678444,8.588200,2.012100,1.917,6.491,95.43
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6,4,2312.0,30.953,3.469333,12.115556,2.536900,2.523,6.842,95.43
2,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6,4,396.0,27.437,0.995125,9.098222,1.855889,1.115,5.767,95.43
3,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6,4,1046.0,28.251,1.400000,9.678000,1.610500,0.304,6.332,95.43
4,Mytopia,Palo Alto,USA,34.007967,-118.489681,6,4,357.0,10.506,1.598714,3.441222,1.091200,0.777,1.982,95.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,BAM Studios,Chicago,USA,41.893622,-87.627423,8,2,231.0,24.437,0.651333,0.612875,0.713300,0.891,0.935,77.35
228,Plugola,Chicago,USA,41.907542,-87.662990,8,2,540.0,21.131,1.614700,1.489750,1.250700,0.964,5.169,77.35
229,Midway Games,Chicago,USA,41.879535,-87.624333,8,2,25.0,25.302,0.777222,0.391857,0.488400,0.565,1.560,77.35
230,Blizzard Entertainment,Irvine,USA,33.669500,-117.822200,1,1,743.0,3.813,2.669778,5.932900,2.634100,0.587,4.248,76.38


In [62]:
#I will scrape the website to get - City + Cost of living
url = "https://www.numbeo.com/cost-of-living/city_price_rankings?itemId=105"
html = requests.get(url)
soup = BeautifulSoup(html.content, "html.parser")
tags = soup.find_all("tr", attrs={"class": "tr_standard"})
tags

[<tr class="tr_standard"><td style="white-space: nowrap; vertical-align: top">
 1.
 </td><td style="vertical-align: top">
 Zug, Switzerland</td><td style="width: 200px; vertical-align: middle">
 <span style="display: inline-block; width: 100%; background-color: #7070ff; height: 0.7em;"></span></td><td style="vertical-align: top; text-align: right">
 7,401.62 $</td></tr>,
 <tr class="tr_standard"><td style="white-space: nowrap; vertical-align: top">
 2.
 </td><td style="vertical-align: top">
 San Jose, CA, United States</td><td style="width: 200px; vertical-align: middle">
 <span style="display: inline-block; width: 91%; background-color: #7070ff; height: 0.7em;"></span></td><td style="vertical-align: top; text-align: right">
 6,728.26 $</td></tr>,
 <tr class="tr_standard"><td style="white-space: nowrap; vertical-align: top">
 3.
 </td><td style="vertical-align: top">
 San Francisco, CA, United States</td><td style="width: 200px; vertical-align: middle">
 <span style="display: inline-bl

In [63]:
nl = []
for e in tags:
    nl.append(e.getText())
nl2 = []
for e in nl:
    e = e.split("\n")
    nl2.append(e)
nl2
nl3=[]
nl4=[]
for i in nl2:
    nl3.append(i[3])
    nl4.append(i[5][:-5])
avg_wage = {"city":nl3, "avg_wage":nl4}
avg_wage_df = pd.DataFrame(avg_wage)
avg_wage_df["city"] = [x.split(",")[0] for x in avg_wage_df["city"]]
avg_wage_df

,city,avg_wage
0,Zug,"7,401"
1,San Jose,"6,728"
2,San Francisco,"6,349"
3,Irvine,"6,306"
4,Seattle,"6,263"
...,...,...
508,Khartoum,141
509,Multan,127
510,Abidjan,126
511,Damascus,50


In [64]:
avg_wage_df.loc[avg_wage_df["city"] == "Cambridge"]

,city,avg_wage
103,Cambridge,"3,182"


In [69]:
avg_wage_df.loc[avg_wage_df["city"] == "San Diego"]

,city,avg_wage
11,San Diego,"5,695"


In [70]:
# The neighbourhoods will have same cost of living as the main city-->> i will add it to the df to join it!
new_row1 = {'city':'Palo Alto', 'avg_wage':6349}
new_row2 = {'city':'San Mateo', 'avg_wage':6349}
new_row3 = {'city':'Burlingame', 'avg_wage':6349}
new_row5 = {'city':'Mountain View', 'avg_wage':6349}
new_row6 = {'city':'Santa Monica', 'avg_wage':5420}
new_row8 = {'city':'Carlsbad', 'avg_wage':5695}

#append row to the dataframe
avg_wage_df = avg_wage_df.append(new_row1, ignore_index=True)
avg_wage_df = avg_wage_df.append(new_row2, ignore_index=True)
avg_wage_df = avg_wage_df.append(new_row3, ignore_index=True)
avg_wage_df = avg_wage_df.append(new_row5, ignore_index=True)
avg_wage_df = avg_wage_df.append(new_row6, ignore_index=True)
avg_wage_df = avg_wage_df.append(new_row8, ignore_index=True)
avg_wage_df

/var/folders/7s/v9wty4qj3t1cw493y3p29dbc0000gn/T/ipykernel_55181/1335226506.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  avg_wage_df = avg_wage_df.append(new_row1, ignore_index=True)
/var/folders/7s/v9wty4qj3t1cw493y3p29dbc0000gn/T/ipykernel_55181/1335226506.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  avg_wage_df = avg_wage_df.append(new_row2, ignore_index=True)
/var/folders/7s/v9wty4qj3t1cw493y3p29dbc0000gn/T/ipykernel_55181/1335226506.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  avg_wage_df = avg_wage_df.append(new_row3, ignore_index=True)
/var/folders/7s/v9wty4qj3t1cw493y3p29dbc0000gn/T/ipykernel_55181/1335226506.py:13: FutureWarning: The frame.append method is deprecated and will be removed from

,city,avg_wage
0,Zug,"7,401"
1,San Jose,"6,728"
2,San Francisco,"6,349"
3,Irvine,"6,306"
4,Seattle,"6,263"
...,...,...
514,San Mateo,6349
515,Burlingame,6349
516,Mountain View,6349
517,Santa Monica,5420


In [72]:
cond = avg_wage_df.city == 'Cambridge'
avg_wage_df.avg_wage[cond] = 5177
avg_wage_df.loc[avg_wage_df["city"] == "Cambridge"]

,city,avg_wage
103,Cambridge,5177


In [77]:
a =[]
for x in avg_wage_df["avg_wage"]:
    if type(x)==str:
        x= x.replace(",","")
    a.append(x)
avg_wage_df["avg_wage"] = a


In [78]:
avg_wage_df

,city,avg_wage
0,Zug,7401
1,San Jose,6728
2,San Francisco,6349
3,Irvine,6306
4,Seattle,6263
...,...,...
514,San Mateo,6349
515,Burlingame,6349
516,Mountain View,6349
517,Santa Monica,5420


In [79]:
df=df.merge(avg_wage_df, left_on='city', right_on='city')
df

,name,city,country,latitude,longitude,nº design comp,nº tech (>$1M),dist to starbucks (m),dist to airport (km),average dist to clubs (km),average dist to vegan rest. (km),average dist to schools (km),dist. to closes basketball court (km),dist. to closest canine salon (km),cost living index,avg_wage
0,Lala,Palo Alto,USA,37.451151,-122.154369,6,4,729.0,27.550,1.678444,8.588200,2.012100,1.917,6.491,95.43,6349
1,PodTech,Palo Alto,USA,37.408256,-122.154176,6,4,2312.0,30.953,3.469333,12.115556,2.536900,2.523,6.842,95.43,6349
2,SonicSwap,Palo Alto,USA,37.444487,-122.163979,6,4,396.0,27.437,0.995125,9.098222,1.855889,1.115,5.767,95.43,6349
3,MyMiniLife,Palo Alto,USA,37.437328,-122.159928,6,4,1046.0,28.251,1.400000,9.678000,1.610500,0.304,6.332,95.43,6349
4,Mytopia,Palo Alto,USA,34.007967,-118.489681,6,4,357.0,10.506,1.598714,3.441222,1.091200,0.777,1.982,95.43,6349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,BAM Studios,Chicago,USA,41.893622,-87.627423,8,2,231.0,24.437,0.651333,0.612875,0.713300,0.891,0.935,77.35,4777
275,Plugola,Chicago,USA,41.907542,-87.662990,8,2,540.0,21.131,1.614700,1.489750,1.250700,0.964,5.169,77.35,4777
276,Midway Games,Chicago,USA,41.879535,-87.624333,8,2,25.0,25.302,0.777222,0.391857,0.488400,0.565,1.560,77.35,4777
277,Blizzard Entertainment,Irvine,USA,33.669500,-117.822200,1,1,743.0,3.813,2.669778,5.932900,2.634100,0.587,4.248,76.38,6306
